In [5]:
%qtconsole

In [1]:
class BibItem:
    
    def __init__(self, text):
        
        # Find the keywords avaialble.
        # For each key in keywords, cut out the line and remove bounding braces.
        
        self.text = text
        self.getKeys()
        self.writeCiteKey()
        
        return

    # Create a dictionary with all the keywords and their values from the bibitem.
    def getKeys(self):
        self.keywords = self.parseKeyWords()
        self.keys = {}
        for key in self.keywords:
            self.keys[key] = self.parseKeyValue(key)
        return
    
    
    # Find the entry type, e.g. arcticle, or inbook. Not currently used.
    def parseType(self):
        self.itemtype = None
        for i in range(1, len(self.text[0])):
            if self.text[0][i] == u'{':
                self.itemtype = self.text[0][1:i]
                break
        return

    
    # Parse the information for the given key.
    def parseKeyValue(self, key):
        
        # Use the special author function.
        if key == 'author':
            return self.parseAuthors()

        # Find the line with the key in.
        for l, line in enumerate(self.text):
            if key in line.lower():
                break
                
        # Parse the appropriate section from the keyword. Strip the curly braces.
        return self.stripBraces(self.getFullLine(l))

        
    # Find the different keys used in the bibliography item.
    def parseKeyWords(self):
        return [l.split('=')[0].split()[0].lower() for l in self.text if '=' in l]

    
    # Find the lines containing the author list, then parse the string
    # of author names, returning those in braces.
    def parseAuthors(self):
        for i, line in enumerate(self.text):
            if 'author' in line.lower():
                break
        lb, rb = 0, 0
        for l, line in enumerate(self.text[i:]):
            lb += line.count(u'{')
            rb += line.count(u'}')
            if lb == rb:
                break
        s = ''.join([l for l in self.text[i:i+l+1]])
        s = s.replace('\n', '')
        names = []
        lb = 0
        rb = 0
        i = 0
        for c, char in enumerate(s):
            if char == u'{':
                lb += 1
                i = c + 1
            elif char == u'}':
                rb += 1
                if (lb - 1 == rb and rb > 0):
                    names.append(self.stripString(s[i:c]))
                    i = c
        return names

    
    # Return the joined strings for one keyword entry.
    def getFullLine(self, i):
        lb, rb = 0, 0
        for l, line in enumerate(self.text[i:]):
            lb += line.count(u'{')
            rb += line.count(u'}')
            if lb == rb:
                break
        s = ''.join([l for l in self.text[i:i+l+1]])
        return s.split('=')[-1]
    

    # Strip a string down to the braces.
    def stripBraces(self, s):
        if (u'{' not in s and u'}' not in s):
            return self.stripString(s).strip()
        for i in range(len(s)):
            if s[i] == u'{':
                i += 1
                break
        for j in range(1, len(s)):
            if s[-j] == u'}':
                j *= -1
                break
        return s[i:j]

    
    # Remove spurious characters in a string.
    def stripString(self, s):
        s = s.replace(u'\\', '')
        s = s.replace(u'"', '')
        s = s.replace(u'{', '')
        s = s.replace(u'}', '')
        s = s.replace(u'`', '')
        s = s.replace(u'\n', '')
        s = s.replace(u'\t', '')
        s = s.replace(u'=', '')
        s = s.replace(',', '')
        return s
    
    
    # Replace the citekey with a standard format: 1 author, author_year; 2 authors, author1_author_2_year and 3+ authors: author1_ea_year
    def writeCiteKey(self):
        if len(self.keys['author']) == 1:
            self.citekey = '%s_%s' % (self.keys['author'][0], self.keys['year'])
        elif len(self.keys['author']) == 2:
            self.citekey = '%s_%s_%s' % (self.keys['author'][0], self.keys['author'][1], self.keys['year'])
        else:
            self.citekey = '%s_ea_%s' % (self.keys['author'][0], self.keys['year'])
        return self.citekey

In [59]:
import string
import numpy as np

class Bibliography:
    
    def __init__(self, filename='bibliography.bib'):
        
        # Upon intialisation, read in the file as a text file and parse into BibItems.
        # Remove all the lines which are empty.
        
        self.filename = filename
        with open(filename) as f:
            self.bib = f.readlines()
            self.bib = [l for l in self.bib if l != '\n']
            
        # Declare the ordering of the months and an alphabet for citekey suffixes.
        
        self.months = {}
        months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 
                  'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
        for m, month in enumerate(months):
            self.months[month]= m
        self.alphabet = list(string.ascii_lowercase)
            
        # Parse into individual bibliography items. Each item becomes a BibItem.
        # Each BibItem gets assigned a citekey, check that there are no duplicates. If so, 
        # append the citekey with a, b, c etc.
        
        self.parseBibItems()
        self.uniqueCiteKeys()
        self.updateCiteKeys()     
        
        print 'All citekeys updated. Ready to write.'
        
        return
    
    # Parse the different bib items.
    def parseBibItems(self):
        self.bibitems = []  
        lb = 0
        rb = 0
        article = 0
        for l, line in enumerate(self.bib):
            lb += line.count(u'{')
            rb += line.count(u'}')
            if lb == rb:
                self.bibitems.append(BibItem(self.bib[article:l+1]))
                lb = 0
                rb = 0
                article = l + 1
        print 'Found %d bibliography items.' % len(self.bibitems)
        return
    
    # Check there are no duplicate citekeys. If they are, append them a, b, c etc. depending on month published.
    def uniqueCiteKeys(self):
        self.citekeys = [bibitem.citekey for bibitem in self.bibitems]
        if len(self.citekeys) != len(set(self.citekeys)):
            duplicate_citekeys = self.findDuplicates()
            for ck in duplicate_citekeys:
                
                # Return the BibItems which are duplicates. 
                # From their month value assign a number and order them.
                # Once ordered, append the appropriate suffix.
                
                bibitems = [bi for bi in self.bibitems if bi.citekey == ck]
                suffix = np.array([self.months[bi.keys['month']] 
                                   for bi in self.bibitems if bi.citekey == ck]).argsort()
                for b, bi in enumerate(bibitems):
                    bi.citekey += self.alphabet[suffix[b]]          
        self.citekeys = [bibitem.citekey for bibitem in self.bibitems]
        return 
    
    # Find the duplicate citekey values.
    def findDuplicates(self):
        seen, dupl = set(), set()
        for citekey in self.citekeys:
            if citekey in seen:
                dupl.add(citekey)
            else:
                seen.add(citekey)
        return list(dupl)
    
    # Change the citekey with the shortened, unique values.
    def updateCiteKeys(self):
        for bibitem in self.bibitems:
            bibitem.text[0] = bibitem.text[0].split(u'{')[0] 
            bibitem.text[0] += '{' + bibitem.citekey + ',\n'
        return
    
    def writeBibliography(self, fileout=None):
        
        if fileout is None:
            fileout = self.filename.split('.')[0] + '_updated.bib'
        
        # Write in alphabetical order somehow.
        
        with open(fileout, 'w') as fo:
            for bibitem in self.bibitems:
                for line in bibitem.text:
                    fo.write(line)
                fo.write('\n')
        
        return

In [60]:
test = Bibliography()

Found 5 bibliography items.
All citekeys updated. Ready to write.


In [61]:
test.writeBibliography()

In [ ]:
test.bib_items[0].authors